In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/25 21:40:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [91]:
# To create spark dataframe, 
#.      u need 
#              list of dicts, list of lists, list of Row, list of string

In [197]:
l = [('Alice', ),("wad",),(1,)]
spark.createDataFrame(l,'name string').show()

+-----+
| name|
+-----+
|Alice|
|  wad|
|    1|
+-----+



In [20]:
l= [(2,"Alice"),(1,"mani")]
spark.createDataFrame(l,"no int,name string").show(

+---+-----+
| no| name|
+---+-----+
|  2|Alice|
|  1| mani|
+---+-----+



## Overview of Row

In [24]:
users = [(1,"scott"),(2,"donald"),(3,"elvis")]

df = spark.createDataFrame(users,'id int,name string')

df.show()

+---+------+
| id|  name|
+---+------+
|  1| scott|
|  2|donald|
|  3| elvis|
+---+------+



In [27]:
# collect() converts dataframe to python list

print(df.collect())

type(df.collect())

[Row(id=1, name='scott'), Row(id=2, name='donald'), Row(id=3, name='elvis')]


list

In [28]:
from pyspark.sql import Row

In [29]:
help(Row)

Help on class Row in module pyspark.sql.types:

class Row(builtins.tuple)
 |  Row(*args, **kwargs)
 |  
 |  A row in :class:`DataFrame`.
 |  The fields in it can be accessed:
 |  
 |  * like attributes (``row.key``)
 |  * like dictionary values (``row[key]``)
 |  
 |  ``key in row`` will search through row keys.
 |  
 |  Row can be used to create a row object by using named arguments.
 |  It is not allowed to omit a named argument to represent that the value is
 |  None or missing. This should be explicitly set to None in this case.
 |  
 |  .. versionchanged:: 3.0.0
 |      Rows created from named arguments no longer have
 |      field names sorted alphabetically and will be ordered in the position as
 |      entered.
 |  
 |  Examples
 |  --------
 |  >>> row = Row(name="Alice", age=11)
 |  >>> row
 |  Row(name='Alice', age=11)
 |  >>> row['name'], row['age']
 |  ('Alice', 11)
 |  >>> row.name, row.age
 |  ('Alice', 11)
 |  >>> 'name' in row
 |  True
 |  >>> 'wrong_key' in row
 |  Fa

In [51]:
# Row with key word arguments

row = Row(name="Alice", age=11)

row.name

'Alice'

In [41]:
# Row with  arguments


row2 = Row("Alice", 11)

row2

<Row('Alice', 11)>

In [49]:
row2 = row2("mani",2000)

In [50]:
row2.Alice

'mani'

## list of list to Spark Dataframe using Row

In [88]:
users = [[1,"mani"],[2,"jaya"]]

# without using ROW

spark.createDataFrame(users,'id int,name string')

DataFrame[id: int, name: string]

In [90]:
# with ROW

users = [ Row(*row) for row in users]

## or

users = [ Row(row[0],row[1]) for row in users]

print(users)

spark.createDataFrame(users,'id int,name string').show()

[<Row(1, 'mani')>, <Row(2, 'jaya')>]
+---+----+
| id|name|
+---+----+
|  1|mani|
|  2|jaya|
+---+----+



In [89]:
def dummy(*arg,**kwargs):
    print(arg,type(arg),kwargs)
    
dummy(1)
dummy(1,"hello")
list1= [1,"hello"]
dummy(list1)       # ------ whole list as a single argument
dummy(*list1)      # ------- list as two arguments

d= {"1":2,"3":4,"5":6}
dummy(*d)           # ------- passing only keys
dummy(**d)          # ----- passing both key and values to kwargs arguments

(1,) <class 'tuple'> {}
(1, 'hello') <class 'tuple'> {}
([1, 'hello'],) <class 'tuple'> {}
(1, 'hello') <class 'tuple'> {}
('1', '3', '5') <class 'tuple'> {}
() <class 'tuple'> {'1': 2, '3': 4, '5': 6}


In [120]:
## list of tuple into spark dataframe using Row

users2 =[(1,"mani"),(2,"jaya"),(3,"mahesh")]

users2 = [Row(*row) for row in users2]

print(users2)

print(spark.createDataFrame(users2).show())


[Row(id=1, name='mani'), Row(id=2, name='jaya'), Row(id=3, name='mahesh')]
+---+------+
| id|  name|
+---+------+
|  1|  mani|
|  2|  jaya|
|  3|mahesh|
+---+------+

None


## list of Dicts into SparkDataFrame

In [65]:
# inferring schema from dict is going to be deprecated, we have to use Row
users_list = [
    {"username":"mani","id":2},
    {"username":"divya","id":3},
    {"username":"ravali","id":4}
]

from pyspark.sql import Row

schema = [Row(**row) for row in users_list]

# convert each one to row and create dataframe

df = spark.createDataFrame(schema)

df.show()

+--------+---+
|username| id|
+--------+---+
|    mani|  2|
|   divya|  3|
|  ravali|  4|
+--------+---+



In [143]:
schema = [Row(*row.values()) for row in users_list]

# *row only pass the keys of dictionary

# convert each one to row and create dataframe

spark.createDataFrame(schema).show()

+------+---+
|    _1| _2|
+------+---+
|  mani|  2|
| divya|  3|
|ravali|  4|
+------+---+



In [83]:
# schema = [Row(row.values()) for row in users_list]
schema = [Row(*row.items()) for row in users_list]
schema

spark.createDataFrame(schema).show()

+------------------+-------+
|                _1|     _2|
+------------------+-------+
|  {username, mani}|{id, 2}|
| {username, divya}|{id, 3}|
|{username, ravali}|{id, 4}|
+------------------+-------+



In [132]:
users_list[1].values()

dict_values(['divya', 3])

In [87]:
for row in users_list:
    print(*row.items())

('username', 'mani') ('id', 2)
('username', 'divya') ('id', 3)
('username', 'ravali') ('id', 4)


## Basic Data Types

In [147]:
df.columns

['username', 'id']

In [149]:
df.dtypes
# to get both column names and datatypes

[('username', 'string'), ('id', 'bigint')]

In [150]:
df.printSchema()

root
 |-- username: string (nullable = true)
 |-- id: long (nullable = true)



## specifying the Schema as String for list of dict

In [106]:
users_schema = '''
   username STRING,
   id BIGINT
'''

spark.createDataFrame(users_list,users_schema).show()

+--------+---+
|username| id|
+--------+---+
|    mani|  2|
|   divya|  3|
|  ravali|  4|
+--------+---+



In [165]:
spark.createDataFrame(users_list,schema = users_schema)

DataFrame[username STRING: bigint, id BIGINT: string]

## SCHEMA using a list

In [107]:
users_schema = [
    'username',
    'id'
]

# with datatype with column name, the datatype is also indulged in the column names in dataframe
#        as the list should contain cloumn names only  
users_schema2 = [
    'username INT',
    'id STRING'
] 


print(spark.createDataFrame(users_list,schema = users_schema).show())

print(spark.createDataFrame(users_list,schema = users_schema2).show())

+--------+------+
|username|    id|
+--------+------+
|       2|  mani|
|       3| divya|
|       4|ravali|
+--------+------+

None
+------------+---------+
|username INT|id STRING|
+------------+---------+
|           2|     mani|
|           3|    divya|
|           4|   ravali|
+------------+---------+

None


In [170]:
users_list

[{'username': 'mani', 'id': 2},
 {'username': 'divya', 'id': 3},
 {'username': 'ravali', 'id': 4}]

## schema with spark types


In [177]:
from pyspark.sql.types import *



In [182]:
# we cant directly use these spark types
# StructType has list of fields made of structField

# StructType  is the data type representing a Row.

# struct1 = StructType().add("f1", StringType(), True).add("f2", StringType(), True, None)
# struct2 = StructType([StructField("f1", StringType(), True),
#     StructField("f2", StringType(), True, None)])
# struct1 == struct2
# True

fields = StructType([
    StructField("username",StringType()),
    StructField("id",IntegerType())
])

print(spark.createDataFrame(users_list,fields))

spark.createDataFrame(users_list,fields).show()

DataFrame[username: string, id: int]
+--------+---+
|username| id|
+--------+---+
|    mani|  2|
|   divya|  3|
|  ravali|  4|
+--------+---+



In [183]:
type(fields)

pyspark.sql.types.StructType


## Spark DataFrame using pandas DataFrame

In [9]:
users_list = [
    {'username': 'mani', 'id': 2},
 {'username': 'divya', 'id': 3},
 {'username': 'ravali', 'id': 4}
]

import pandas as pd

df1 = pd.DataFrame(users_list)

df1.head()
# pandas add NaN in empty spaces
# which will be helpful in creating spark dataframe as we dont have to add the values explicitly in missed columns

,username,id
0,mani,2
1,divya,3
2,ravali,4


In [11]:
spark.createDataFrame(df1).show()

+--------+---+
|username| id|
+--------+---+
|    mani|  2|
|   divya|  3|
|  ravali|  4|
+--------+---+



## special types from spark

In [12]:
# ARRAY
# STRUCT
# MAP

# Python structures such as list and dict can be implicitly converted to spark array and map respectively
# we need spark APIs to convert python DataStructures to STRUCT type

In [46]:

import datetime
from pyspark.sql import Row

users = [
    {
        "id": 1,
        "first_name": "Corrie",
        "last_name": "Van den Oord",
        "email": "cvandenoord0@etsy.com",
        "phone_numbers": ["+1 234 567 8901", "+1 234 567 8911"],
        "is_customer": True,
        "amount_paid": 1000.55,
        "customer_from": datetime.date(2021, 1, 15),
        "last_updated_ts": datetime.datetime(2021, 2, 10, 1, 15, 0)
    },
    {
        "id": 2,
        "first_name": "Nikolaus",
        "last_name": "Brewitt",
        "email": "nbrewitt1@dailymail.co.uk",
        "phone_numbers": ["+1 234 567 8923", "+1 234 567 8934"],
        "is_customer": True,
        "amount_paid": 900.0,
        "customer_from": datetime.date(2021, 2, 14),
        "last_updated_ts": datetime.datetime(2021, 2, 18, 3, 33, 0)
    },
    {
        "id": 3,
        "first_name": "Orelie",
        "last_name": "Penney",
        "email": "openney2@vistaprint.com",
        "phone_numbers": ["+1 714 512 9752", "+1 714 512 6601"],
        "is_customer": True,
        "amount_paid": 850.55,
        "customer_from": datetime.date(2021, 1, 21),
        "last_updated_ts": datetime.datetime(2021, 3, 15, 15, 16, 55)
    },
    {
        "id": 4,
        "first_name": "Ashby",
        "last_name": "Maddocks",
        "email": "amaddocks3@home.pl",
        "phone_numbers": None,
        "is_customer": False,
        "amount_paid": None,
        "customer_from": None,
        "last_updated_ts": datetime.datetime(2021, 4, 10, 17, 45, 30)
    },
    {
        "id": 5,
        "first_name": "Kurt",
        "last_name": "Rome",
        "email": "krome4@shutterfly.com",
        "phone_numbers": ["+1 817 934 7142"],
        "is_customer": False,
        "amount_paid": None,
        "customer_from": None,
        "last_updated_ts": datetime.datetime(2021, 4, 2, 0, 55, 18)
    }
]

users_df = spark.createDataFrame([Row(**user) for user in users])

users_df.show()

+---+----------+------------+--------------------+--------------------+-----------+-----------+-------------+-------------------+
| id|first_name|   last_name|               email|       phone_numbers|is_customer|amount_paid|customer_from|    last_updated_ts|
+---+----------+------------+--------------------+--------------------+-----------+-----------+-------------+-------------------+
|  1|    Corrie|Van den Oord|cvandenoord0@etsy...|[+1 234 567 8901,...|       true|    1000.55|   2021-01-15|2021-02-10 01:15:00|
|  2|  Nikolaus|     Brewitt|nbrewitt1@dailyma...|[+1 234 567 8923,...|       true|      900.0|   2021-02-14|2021-02-18 03:33:00|
|  3|    Orelie|      Penney|openney2@vistapri...|[+1 714 512 9752,...|       true|     850.55|   2021-01-21|2021-03-15 15:16:55|
|  4|     Ashby|    Maddocks|  amaddocks3@home.pl|                null|      false|       null|         null|2021-04-10 17:45:30|
|  5|      Kurt|        Rome|krome4@shutterfly...|   [+1 817 934 7142]|      false|       

In [16]:
users_df.dtypes

[('id', 'bigint'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('email', 'string'),
 ('phone_numbers', 'array<string>'),
 ('is_customer', 'boolean'),
 ('amount_paid', 'double'),
 ('customer_from', 'date'),
 ('last_updated_ts', 'timestamp')]

In [18]:
users_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone_numbers: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- is_customer: boolean (nullable = true)
 |-- amount_paid: double (nullable = true)
 |-- customer_from: date (nullable = true)
 |-- last_updated_ts: timestamp (nullable = true)



In [19]:
users_df.select("id","phone_numbers").show(truncate=False)

# truncate= False to get all numbers without truncation

+---+----------------------------------+
|id |phone_numbers                     |
+---+----------------------------------+
|1  |[+1 234 567 8901, +1 234 567 8911]|
|2  |[+1 234 567 8923, +1 234 567 8934]|
|3  |[+1 714 512 9752, +1 714 512 6601]|
|4  |null                              |
|5  |[+1 817 934 7142]                 |
+---+----------------------------------+



In [20]:
users_df.columns

['id',
 'first_name',
 'last_name',
 'email',
 'phone_numbers',
 'is_customer',
 'amount_paid',
 'customer_from',
 'last_updated_ts']

In [51]:
from pyspark.sql.functions import explode

# Explode - it is to explode a column with a Array type into a multiple records.
#         - Returns a new row for each element in the given array or map

# withColumn - Returns a new DataFrame by adding a column or replacing the existing column that has the same name.

users_df. \
    withColumn("phone_number",explode("phone_numbers")).\
    show(truncate=False)
               

+---+----------+------------+-------------------------+----------------------------------+-----------+-----------+-------------+-------------------+---------------+
|id |first_name|last_name   |email                    |phone_numbers                     |is_customer|amount_paid|customer_from|last_updated_ts    |phone_number   |
+---+----------+------------+-------------------------+----------------------------------+-----------+-----------+-------------+-------------------+---------------+
|1  |Corrie    |Van den Oord|cvandenoord0@etsy.com    |[+1 234 567 8901, +1 234 567 8911]|true       |1000.55    |2021-01-15   |2021-02-10 01:15:00|+1 234 567 8901|
|1  |Corrie    |Van den Oord|cvandenoord0@etsy.com    |[+1 234 567 8901, +1 234 567 8911]|true       |1000.55    |2021-01-15   |2021-02-10 01:15:00|+1 234 567 8911|
|2  |Nikolaus  |Brewitt     |nbrewitt1@dailymail.co.uk|[+1 234 567 8923, +1 234 567 8934]|true       |900.0      |2021-02-14   |2021-02-18 03:33:00|+1 234 567 8923|
|2  |Nikol

In [52]:
from pyspark.sql.functions import col

In [27]:

# to make individual columns from an array with aliases

users_df.\
       select("id",col('phone_numbers')[0].alias("mobile"),col("phone_numbers")[1].alias("home")).show()

+---+---------------+---------------+
| id|         mobile|           home|
+---+---------------+---------------+
|  1|+1 234 567 8901|+1 234 567 8911|
|  2|+1 234 567 8923|+1 234 567 8934|
|  3|+1 714 512 9752|+1 714 512 6601|
|  4|           null|           null|
|  5|+1 817 934 7142|           null|
+---+---------------+---------------+



## with explode_outer

In [31]:
from pyspark.sql.functions import explode_outer

# Explode_outer - to get all non-Null and Null values present in exploding column

users_df. \
    withColumn("phone_number",explode_outer("phone_numbers")).\
    show()

+---+----------+------------+--------------------+--------------------+-----------+-----------+-------------+-------------------+---------------+
| id|first_name|   last_name|               email|       phone_numbers|is_customer|amount_paid|customer_from|    last_updated_ts|   phone_number|
+---+----------+------------+--------------------+--------------------+-----------+-----------+-------------+-------------------+---------------+
|  1|    Corrie|Van den Oord|cvandenoord0@etsy...|[+1 234 567 8901,...|       true|    1000.55|   2021-01-15|2021-02-10 01:15:00|+1 234 567 8901|
|  1|    Corrie|Van den Oord|cvandenoord0@etsy...|[+1 234 567 8901,...|       true|    1000.55|   2021-01-15|2021-02-10 01:15:00|+1 234 567 8911|
|  2|  Nikolaus|     Brewitt|nbrewitt1@dailyma...|[+1 234 567 8923,...|       true|      900.0|   2021-02-14|2021-02-18 03:33:00|+1 234 567 8923|
|  2|  Nikolaus|     Brewitt|nbrewitt1@dailyma...|[+1 234 567 8923,...|       true|      900.0|   2021-02-14|2021-02-18 03:3

## Map Type Columns in Spark DataFrames

In [53]:
import datetime
users = [
    {
        "id": 1,
        "first_name": "Corrie",
        "last_name": "Van den Oord",
        "email": "cvandenoord0@etsy.com",
        "phone_numbers": {"mobile": "+1 234 567 8901", "home": "+1 234 567 8911"},
        "is_customer": True,
        "amount_paid": 1000.55,
        "customer_from": datetime.date(2021, 1, 15),
        "last_updated_ts": datetime.datetime(2021, 2, 10, 1, 15, 0)
    },
    {
        "id": 2,
        "first_name": "Nikolaus",
        "last_name": "Brewitt",
        "email": "nbrewitt1@dailymail.co.uk",
        "phone_numbers": {"mobile": "+1 234 567 8923", "home": "+1 234 567 8934"},
        "is_customer": True,
        "amount_paid": 900.0,
        "customer_from": datetime.date(2021, 2, 14),
        "last_updated_ts": datetime.datetime(2021, 2, 18, 3, 33, 0)
    },
    {
        "id": 3,
        "first_name": "Orelie",
        "last_name": "Penney",
        "email": "openney2@vistaprint.com",
        "phone_numbers": {"mobile": "+1 714 512 9752", "home": "+1 714 512 6601"},
        "is_customer": True,
        "amount_paid": 850.55,
        "customer_from": datetime.date(2021, 1, 21),
        "last_updated_ts": datetime.datetime(2021, 3, 15, 15, 16, 55)
    },
    {
        "id": 4,
        "first_name": "Ashby",
        "last_name": "Maddocks",
        "email": "amaddocks3@home.pl",
        "phone_numbers": None,
        "is_customer": False,
        "amount_paid": None,
        "customer_from": None,
        "last_updated_ts": datetime.datetime(2021, 4, 10, 17, 45, 30)
    },
    {
        "id": 5,
        "first_name": "Kurt",
        "last_name": "Rome",
        "email": "krome4@shutterfly.com",
        "phone_numbers": {"mobile": "+1 817 934 7142"},
        "is_customer": False,
        "amount_paid": None,
        "customer_from": None,
        "last_updated_ts": datetime.datetime(2021, 4, 2, 0, 55, 18)
    }
]


# phone numbers are in dictionary

users_df = spark.createDataFrame([Row(**user) for user in users])

users_df.show(truncate=False)


+---+----------+------------+-------------------------+----------------------------------------------------+-----------+-----------+-------------+-------------------+
|id |first_name|last_name   |email                    |phone_numbers                                       |is_customer|amount_paid|customer_from|last_updated_ts    |
+---+----------+------------+-------------------------+----------------------------------------------------+-----------+-----------+-------------+-------------------+
|1  |Corrie    |Van den Oord|cvandenoord0@etsy.com    |{mobile -> +1 234 567 8901, home -> +1 234 567 8911}|true       |1000.55    |2021-01-15   |2021-02-10 01:15:00|
|2  |Nikolaus  |Brewitt     |nbrewitt1@dailymail.co.uk|{mobile -> +1 234 567 8923, home -> +1 234 567 8934}|true       |900.0      |2021-02-14   |2021-02-18 03:33:00|
|3  |Orelie    |Penney      |openney2@vistaprint.com  |{mobile -> +1 714 512 9752, home -> +1 714 512 6601}|true       |850.55     |2021-01-21   |2021-03-15 15:16:55

In [34]:
users_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone_numbers: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- is_customer: boolean (nullable = true)
 |-- amount_paid: double (nullable = true)
 |-- customer_from: date (nullable = true)
 |-- last_updated_ts: timestamp (nullable = true)



In [36]:
users_df.select("id","phone_numbers").show(truncate=False)

# the dict is represe as key -> value pairs

+---+----------------------------------------------------+
|id |phone_numbers                                       |
+---+----------------------------------------------------+
|1  |{mobile -> +1 234 567 8901, home -> +1 234 567 8911}|
|2  |{mobile -> +1 234 567 8923, home -> +1 234 567 8934}|
|3  |{mobile -> +1 714 512 9752, home -> +1 714 512 6601}|
|4  |null                                                |
|5  |{mobile -> +1 817 934 7142}                         |
+---+----------------------------------------------------+



In [38]:
users_df.dtypes

# phone_numbers is a type of map

[('id', 'bigint'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('email', 'string'),
 ('phone_numbers', 'map<string,string>'),
 ('is_customer', 'boolean'),
 ('amount_paid', 'double'),
 ('customer_from', 'date'),
 ('last_updated_ts', 'timestamp')]

In [39]:

# To select only mobile numbers without alias

users_df.select("id",col("phone_numbers")["mobile"]).show()

+---+---------------------+
| id|phone_numbers[mobile]|
+---+---------------------+
|  1|      +1 234 567 8901|
|  2|      +1 234 567 8923|
|  3|      +1 714 512 9752|
|  4|                 null|
|  5|      +1 817 934 7142|
+---+---------------------+



In [62]:
# to get separate columns of map type column with alias
users_df.select('id', explode('phone_numbers').alias('key_new', 'value_new')).show()

+---+-------+---------------+
| id|key_new|      value_new|
+---+-------+---------------+
|  1| mobile|+1 234 567 8901|
|  1|   home|+1 234 567 8911|
|  2| mobile|+1 234 567 8923|
|  2|   home|+1 234 567 8934|
|  3| mobile|+1 714 512 9752|
|  3|   home|+1 714 512 6601|
|  5| mobile|+1 817 934 7142|
+---+-------+---------------+



In [54]:
from pyspark.sql.functions import explode

users_df.select("id",explode("phone_numbers")).show()

# here phone_numbers is a map

# we can also rename the columns as we needed

+---+------+---------------+
| id|   key|          value|
+---+------+---------------+
|  1|mobile|+1 234 567 8901|
|  1|  home|+1 234 567 8911|
|  2|mobile|+1 234 567 8923|
|  2|  home|+1 234 567 8934|
|  3|mobile|+1 714 512 9752|
|  3|  home|+1 714 512 6601|
|  5|mobile|+1 817 934 7142|
+---+------+---------------+



In [57]:
# Renaming the columns of above DataFrame

users_df.select("id",explode("phone_numbers")).\
            withColumnRenamed("key","phone_type").\
            withColumnRenamed("value","phone_number").\
            show()

+---+----------+---------------+
| id|phone_type|   phone_number|
+---+----------+---------------+
|  1|    mobile|+1 234 567 8901|
|  1|      home|+1 234 567 8911|
|  2|    mobile|+1 234 567 8923|
|  2|      home|+1 234 567 8934|
|  3|    mobile|+1 714 512 9752|
|  3|      home|+1 714 512 6601|
|  5|    mobile|+1 817 934 7142|
+---+----------+---------------+



In [59]:
# To get all columns , use * 

users_df.select("*",explode("phone_numbers")).\
            withColumnRenamed("key","phone_type").\
            withColumnRenamed("value","phone_number").\
            drop("phone_numbers").\
            show()

+---+----------+------------+--------------------+-----------+-----------+-------------+-------------------+----------+---------------+
| id|first_name|   last_name|               email|is_customer|amount_paid|customer_from|    last_updated_ts|phone_type|   phone_number|
+---+----------+------------+--------------------+-----------+-----------+-------------+-------------------+----------+---------------+
|  1|    Corrie|Van den Oord|cvandenoord0@etsy...|       true|    1000.55|   2021-01-15|2021-02-10 01:15:00|    mobile|+1 234 567 8901|
|  1|    Corrie|Van den Oord|cvandenoord0@etsy...|       true|    1000.55|   2021-01-15|2021-02-10 01:15:00|      home|+1 234 567 8911|
|  2|  Nikolaus|     Brewitt|nbrewitt1@dailyma...|       true|      900.0|   2021-02-14|2021-02-18 03:33:00|    mobile|+1 234 567 8923|
|  2|  Nikolaus|     Brewitt|nbrewitt1@dailyma...|       true|      900.0|   2021-02-14|2021-02-18 03:33:00|      home|+1 234 567 8934|
|  3|    Orelie|      Penney|openney2@vistapri..

In [64]:
# withColumn - Returns a new DataFrame by adding a column or replacing the existing column that has the same name.

users_df. \
    withColumn('phone_numbers_home', col('phone_numbers')['home']). \
    withColumn('phone_numbers_mobile', col('phone_numbers')['mobile'] ). \
    drop("phone_numbers").\
    show()

+---+----------+------------+--------------------+-----------+-----------+-------------+-------------------+------------------+--------------------+
| id|first_name|   last_name|               email|is_customer|amount_paid|customer_from|    last_updated_ts|phone_numbers_home|phone_numbers_mobile|
+---+----------+------------+--------------------+-----------+-----------+-------------+-------------------+------------------+--------------------+
|  1|    Corrie|Van den Oord|cvandenoord0@etsy...|       true|    1000.55|   2021-01-15|2021-02-10 01:15:00|   +1 234 567 8911|     +1 234 567 8901|
|  2|  Nikolaus|     Brewitt|nbrewitt1@dailyma...|       true|      900.0|   2021-02-14|2021-02-18 03:33:00|   +1 234 567 8934|     +1 234 567 8923|
|  3|    Orelie|      Penney|openney2@vistapri...|       true|     850.55|   2021-01-21|2021-03-15 15:16:55|   +1 714 512 6601|     +1 714 512 9752|
|  4|     Ashby|    Maddocks|  amaddocks3@home.pl|      false|       null|         null|2021-04-10 17:45:3

23/06/26 01:49:13 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 4352152 ms exceeds timeout 120000 ms
23/06/26 01:49:13 WARN SparkContext: Killing executors is not supported by current scheduler.


## Struct type Columns in Spark DataFrames